In [23]:
import pandas as pd

books = pd.read_csv("books_with_final_category.csv")

In [24]:
from transformers import pipeline
classifier = pipeline("text-classification",
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k = None,
                      device = "cpu")

Device set to use cpu


In [25]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [26]:
classifier(books["description"][0])


[[{'label': 'fear', 'score': 0.6548418998718262},
  {'label': 'neutral', 'score': 0.16985182464122772},
  {'label': 'sadness', 'score': 0.11640854179859161},
  {'label': 'surprise', 'score': 0.020700592547655106},
  {'label': 'disgust', 'score': 0.01910073682665825},
  {'label': 'joy', 'score': 0.015161239542067051},
  {'label': 'anger', 'score': 0.00393515033647418}]]

In [27]:
classifier(books["description"][0].split("."))


[[{'label': 'surprise', 'score': 0.7296027541160583},
  {'label': 'neutral', 'score': 0.14038576185703278},
  {'label': 'fear', 'score': 0.06816209107637405},
  {'label': 'joy', 'score': 0.04794240742921829},
  {'label': 'anger', 'score': 0.00915635284036398},
  {'label': 'disgust', 'score': 0.0026284719351679087},
  {'label': 'sadness', 'score': 0.0021221607457846403}],
 [{'label': 'neutral', 'score': 0.449370414018631},
  {'label': 'disgust', 'score': 0.27359241247177124},
  {'label': 'joy', 'score': 0.10908260941505432},
  {'label': 'sadness', 'score': 0.09362703561782837},
  {'label': 'anger', 'score': 0.040478307753801346},
  {'label': 'surprise', 'score': 0.02697017230093479},
  {'label': 'fear', 'score': 0.006879065651446581}],
 [{'label': 'neutral', 'score': 0.6462168097496033},
  {'label': 'sadness', 'score': 0.2427326887845993},
  {'label': 'disgust', 'score': 0.04342268034815788},
  {'label': 'surprise', 'score': 0.028300466015934944},
  {'label': 'joy', 'score': 0.014211482

In [28]:
sentences = books["description"][0].split(".")
predictions = classifier(sentences)
sentences[0]


'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [29]:
predictions[0]


[{'label': 'surprise', 'score': 0.7296027541160583},
 {'label': 'neutral', 'score': 0.14038576185703278},
 {'label': 'fear', 'score': 0.06816209107637405},
 {'label': 'joy', 'score': 0.04794240742921829},
 {'label': 'anger', 'score': 0.00915635284036398},
 {'label': 'disgust', 'score': 0.0026284719351679087},
 {'label': 'sadness', 'score': 0.0021221607457846403}]

In [30]:
import numpy as np

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_average_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.mean(scores) for label, scores in per_emotion_scores.items()}

In [31]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    average_scores = calculate_average_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(average_scores[label])


In [32]:
emotion_scores


{'anger': [np.float64(0.02108500561735127),
  np.float64(0.0887243835721165),
  np.float64(0.03539548278786242),
  np.float64(0.039533850736916065),
  np.float64(0.10958813922479749),
  np.float64(0.04434006807527372),
  np.float64(0.13018580223433673),
  np.float64(0.11615158325399864),
  np.float64(0.030368382188801963),
  np.float64(0.03652489511296153)],
 'disgust': [np.float64(0.06034896583878435),
  np.float64(0.07088199253194034),
  np.float64(0.054111577570438385),
  np.float64(0.04336955479811877),
  np.float64(0.07961069187149405),
  np.float64(0.08300441250737224),
  np.float64(0.3451300992164761),
  np.float64(0.05647664523514157),
  np.float64(0.05007618727783362),
  np.float64(0.054716031067073345)],
 'fear': [np.float64(0.1347652367912815),
  np.float64(0.1727293192059733),
  np.float64(0.0304981991648674),
  np.float64(0.3759149710337321),
  np.float64(0.10391970063210465),
  np.float64(0.034571821502010734),
  np.float64(0.03092675283551216),
  np.float64(0.19363722203

In [33]:
import numpy as np

def normalize_scores(scores_dict):
    scores = np.array(list(scores_dict.values()))
    normalized = scores / scores.sum() if scores.sum() > 0 else scores
    return {label: normalized[i] for i, label in enumerate(scores_dict.keys())}


In [34]:
from tqdm import tqdm
import numpy as np

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_average_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.mean(scores) for label, scores in per_emotion_scores.items()}

def normalize_scores(scores_dict):
    scores = np.array(list(scores_dict.values()))
    normalized = scores / scores.sum() if scores.sum() > 0 else scores
    return {label: normalized[i] for i, label in enumerate(scores_dict.keys())}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    average_scores = calculate_average_emotion_scores(predictions)
    normalized_scores = normalize_scores(average_scores)
    for label in emotion_labels:
        emotion_scores[label].append(normalized_scores[label])


100%|██████████| 6374/6374 [32:59<00:00,  3.22it/s]  


In [35]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn

In [36]:
emotions_df


,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.021085,0.060349,0.134765,0.225031,0.260441,0.180305,0.118024,9780002005883
1,0.088724,0.070882,0.172729,0.108252,0.421635,0.042325,0.095452,9780002261982
2,0.035395,0.054112,0.030498,0.302843,0.460771,0.061337,0.055044,9780006163831
3,0.039534,0.043370,0.375915,0.270091,0.198765,0.041284,0.031043,9780006178736
4,0.109588,0.079611,0.103920,0.114356,0.516222,0.046548,0.029755,9780006280897
...,...,...,...,...,...,...,...,...
6369,0.032901,0.058441,0.025088,0.134224,0.623777,0.034330,0.091239,9788173031014
6370,0.009172,0.007595,0.118272,0.569989,0.245900,0.026647,0.022426,9788179921623
6371,0.015750,0.034882,0.061970,0.141426,0.658565,0.057510,0.029897,9788185300535
6372,0.014982,0.028539,0.011923,0.295203,0.605833,0.020789,0.022731,9789027712059


In [37]:
books = pd.merge(books, emotions_df, on = "isbn13")


In [38]:
books.to_csv("books_with_emotions.csv", index = False)